<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/kaggle_predictFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# kaggle_Predint_Future_Sales
!pip install kaggle
from google.colab import files
files.upload()

In [0]:
import os
os.chdir('/content')
!ls -1ha kaggle.json

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
import os
os.mkdir('/content/predFS/')
os.chdir('/content/predFS/')

In [0]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

In [0]:
os.chdir('/content/predFS/')
!ls

In [0]:
os.mkdir('/content/predFS/input')
!mv item_categories.csv /content/predFS/input
!mv sales_train.csv.zip /content/predFS/input
!mv shops.csv /content/predFS/input
!mv items.csv.zip /content/predFS/input
!mv sample_submission.csv.zip /content/predFS/input
!mv test.csv.zip /content/predFS/input

In [0]:
os.chdir('/content/predFS/input')
!unzip sales_train.csv.zip 
!unzip items.csv.zip
!unzip sample_submission.csv.zip
!unzip test.csv.zip


In [0]:
#https://www.kaggle.com/dlarionov/feature-engineering-xgboost

# feature engineering
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
sys.version_info

In [0]:
items = pd.read_csv('../input/items.csv')
shops = pd.read_csv('../input/shops.csv')
cats = pd.read_csv('../input/item_categories.csv')
train = pd.read_csv('../input/sales_train.csv')
# set index to ID to avoid droping it later
test  = pd.read_csv('../input/test.csv').set_index('ID')

**Data Field**

**ID** - an Id that represents a (Shop, Item) tuple within the test set

**shop_id** - unique identifier of a shop

**item_id** - unique identifier of a product

**item_category_id** - unique identifier of item category

**item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure

**item_price** - current price of an item

**date** - date in format dd/mm/yyyy

**date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

**item_name** - name of item

**shop_name** - name of shop

**item_category_name** - name of item category

In [0]:
train.head()

In [0]:
# outlier 찾기
plt.figure(figsize=(10,4)) # 그래프 판 사이즈
plt.xlim(-100, 3000) # x 좌표 설정
sns.boxplot(x=train.item_cnt_day) # 어떤 샵의 어떤 제품의 어떤 날짜 1일 판매량 박스 플롯으로

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1) # x 좌표 설정, 가격 최소값과 최대값을 직접 입력
sns.boxplot(x=train.item_price) # 제품 가격 박스 플롯

In [0]:
#outlier 제거
train = train[train.item_price<100000]  # 가격 100000 미만
train = train[train.item_cnt_day<1001]  # 하루 판매량 1000 이하

In [0]:
# 결측값 채우기
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
# 가격 외 위 조건의 항목 중 가격이 음의 값이 존재하며, 이를 같은 조건을 가진 다른 케이스들의 중앙값(median)으로 채움
train.loc[train.item_price<0, 'item_price'] = median 
# loc 다음은 행 값, 열 값을 줄 수 있음. 이 케이스의 경우 가격이 음인 케이스는 하나이므로 행조건에 별다른 조건을 걸지 않음

In [0]:
shops

In [0]:
# 같은 가게를 다르게 입력한 케이스 통합, 단순 데이터 중복이 아닌 내용적 중복을 잡은 것이지만 그냥 스킬적으로만 알자
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [0]:
# 
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)